اگر آپ نے ابھی تک نہیں کیا ہے تو درج ذیل نوٹ بکس چلانے کے لیے، آپ کو .env فائل کے اندر `OPENAI_API_KEY` کے طور پر اوپن اے آئی کی سیٹ کرنی ہوگی۔


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

اگلا، ہم ایمبیڈنگ انڈیکس کو پانڈاز ڈیٹا فریم میں لوڈ کرنے جا رہے ہیں۔ ایمبیڈنگ انڈیکس ایک JSON فائل میں محفوظ ہے جس کا نام `embedding_index_3m.json` ہے۔ ایمبیڈنگ انڈیکس میں ہر یوٹیوب ٹرانسکرپٹ کے ایمبیڈنگ شامل ہیں جو اکتوبر 2023 کے آخر تک کے ہیں۔


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

اگلا، ہم ایک فنکشن بنائیں گے جسے `get_videos` کہا جائے گا جو کوئری کے لیے ایمبیڈنگ انڈیکس میں تلاش کرے گا۔ یہ فنکشن ان 5 ویڈیوز کو واپس کرے گا جو کوئری کے سب سے زیادہ مشابہ ہوں۔ فنکشن اس طرح کام کرتا ہے:

1. سب سے پہلے، ایمبیڈنگ انڈیکس کی ایک کاپی بنائی جاتی ہے۔
2. اگلا، کوئری کے لیے ایمبیڈنگ اوپن اے آئی ایمبیڈنگ API کا استعمال کرتے ہوئے حساب کی جاتی ہے۔
3. پھر ایمبیڈنگ انڈیکس میں `similarity` نام کا ایک نیا کالم بنایا جاتا ہے۔ `similarity` کالم میں کوئری ایمبیڈنگ اور ہر ویڈیو سیگمنٹ کی ایمبیڈنگ کے درمیان کوسائن مشابہت شامل ہوتی ہے۔
4. اگلا، ایمبیڈنگ انڈیکس کو `similarity` کالم کی بنیاد پر فلٹر کیا جاتا ہے۔ ایمبیڈنگ انڈیکس کو صرف ان ویڈیوز تک محدود کیا جاتا ہے جن کی کوسائن مشابہت 0.75 یا اس سے زیادہ ہو۔
5. آخر میں، ایمبیڈنگ انڈیکس کو `similarity` کالم کے مطابق ترتیب دیا جاتا ہے اور سب سے اوپر کے 5 ویڈیوز واپس کیے جاتے ہیں۔


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

یہ فنکشن بہت آسان ہے، یہ صرف تلاش کے سوال کے نتائج کو پرنٹ کرتا ہے۔


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. سب سے پہلے، ایمبیڈنگ انڈیکس کو پانڈاز ڈیٹافریم میں لوڈ کیا جاتا ہے۔
2. اگلا، صارف کو ایک سوال درج کرنے کے لیے کہا جاتا ہے۔
3. پھر `get_videos` فنکشن کو کال کیا جاتا ہے تاکہ سوال کے لیے ایمبیڈنگ انڈیکس میں تلاش کی جا سکے۔
4. آخر میں، `display_results` فنکشن کو کال کیا جاتا ہے تاکہ نتائج صارف کو دکھائے جا سکیں۔
5. پھر صارف کو ایک اور سوال درج کرنے کے لیے کہا جاتا ہے۔ یہ عمل اس وقت تک جاری رہتا ہے جب تک صارف `exit` درج نہ کرے۔

![](../../../../translated_images/ur/notebook-search.1e320b9c7fcbb0bc.webp)

آپ سے سوال درج کرنے کو کہا جائے گا۔ ایک سوال درج کریں اور انٹر دبائیں۔ ایپلیکیشن سوال سے متعلق ویڈیوز کی فہرست واپس کرے گی۔ ایپلیکیشن اس جگہ کا لنک بھی فراہم کرے گی جہاں ویڈیو میں سوال کا جواب موجود ہے۔

یہاں کچھ سوالات ہیں جنہیں آپ آزما سکتے ہیں:

- Azure Machine Learning کیا ہے؟
- Convolutional neural networks کیسے کام کرتے ہیں؟
- Neural network کیا ہے؟
- کیا میں Jupyter Notebooks کو Azure Machine Learning کے ساتھ استعمال کر سکتا ہوں؟
- ONNX کیا ہے؟


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**دستخطی دستبرداری**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کے ذریعے ترجمہ کی گئی ہے۔ اگرچہ ہم درستگی کے لیے کوشاں ہیں، براہ کرم اس بات سے آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستیاں ہو سکتی ہیں۔ اصل دستاویز اپنی مادری زبان میں ہی معتبر ماخذ سمجھی جانی چاہیے۔ اہم معلومات کے لیے پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کی ذمہ داری ہم پر عائد نہیں ہوتی۔
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
